In [ ]:
pip install selenium


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 4.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.47qK4hTCPp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.b1qH8F0A9f/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.vb2dxNQqHN/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
# Install chromium browser and driver
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:11 http://archive.ubuntu.com/ubuntu focal InRelease
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:13 http://deb.debian.org/debian-security bus

In [ ]:
# Create Selenium options
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
# Initiate chrome webdriver
from googlesearch import search
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

KEYWORDS = ['ngập nước hồ chí minh', 'ngập nước Sài gòn']
# options = webdriver.ChromeOptions()
# options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
# chrome_driver_binary = r"C:/Users/Max/.wdm/chromedriver/75.0.3770.8/win32/chromedriver.exe"
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = web_driver()


df = pd.DataFrame(columns=['url', 'title', 'content'])

for key in KEYWORDS:
	for link in search(key, lang='vi', stop=5):
		driver.get(link)
		html = driver.page_source

		soup = BeautifulSoup(html)
		title = soup.findAll("title")[0].getText()
		body = soup.find_all("body")[0].getText()

		print(title)


		row = {'url': [link], 'title': [title], 'content': [" ".join(body.split())]}
		df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)
	
	
driver.quit()

df

Tin tức mới nhất ngập ở TP Hồ Chí Minh trên VnExpress
Ngập nước ở Sài Gòn - VnExpress
Những con đường hay ngập vào mùa mưa ở thành phố Hồ Chí Minh
Cập nhật bản đồ ngập nước thành phố mới nhất 2021
Chiều 1-10, TP.HCM mưa tối trời, nhiều tuyến đường ngập quá nửa bánh xe - Tuổi Trẻ Online
Ngập nước ở Sài Gòn - VnExpress
Tin tức mới nhất ngập ở TP Hồ Chí Minh trên VnExpress
Cập nhật bản đồ ngập nước thành phố mới nhất 2021
Sài Gòn ngập nặng
Tổng hợp các đoạn đường bị ngập nước vào mùa mưa tại TPHCM | Shop2banh.vn


,url,title,content
0,https://vnexpress.net/chu-de/ngap-o-tp-ho-chi-...,Tin tức mới nhất ngập ở TP Hồ Chí Minh trên Vn...,"Chủ nhật, 12/2/2023 Mới nhất Tin theo khu vực ..."
1,https://vnexpress.net/topic/ngap-nuoc-o-sai-go...,Ngập nước ở Sài Gòn - VnExpress,"Chủ nhật, 12/2/2023 Mới nhất Tin theo khu vực ..."
2,https://www.bachhoaxanh.com/kinh-nghiem-hay/nh...,Những con đường hay ngập vào mùa mưa ở thành p...,BachhoaXANH là đại siêu thị với hơn 15.000 sản...
3,https://smartrealtors.vn/cap-nhat-ban-do-ngap-...,Cập nhật bản đồ ngập nước thành phố mới nhất 2021,Trang chủ Giới thiệu Giải thưởng Dự án Tin tức...
4,https://tuoitre.vn/chieu-1-10-tp-hcm-mua-toi-t...,"Chiều 1-10, TP.HCM mưa tối trời, nhiều tuyến đ...","var ttostarteamplate = """"; if (document.cookie..."
5,https://vnexpress.net/topic/ngap-nuoc-o-sai-go...,Ngập nước ở Sài Gòn - VnExpress,"Chủ nhật, 12/2/2023 Mới nhất Tin theo khu vực ..."
6,https://vnexpress.net/chu-de/ngap-o-tp-ho-chi-...,Tin tức mới nhất ngập ở TP Hồ Chí Minh trên Vn...,"Chủ nhật, 12/2/2023 Mới nhất Tin theo khu vực ..."
7,https://smartrealtors.vn/cap-nhat-ban-do-ngap-...,Cập nhật bản đồ ngập nước thành phố mới nhất 2021,Trang chủ Giới thiệu Giải thưởng Dự án Tin tức...
8,https://vietnamnet.vn/sai-gon-ngap-nang-tag101...,Sài Gòn ngập nặng,#TenMax_sticky { transition: bottom 1s; positi...
9,https://shop2banh.vn/tu-van/tong-hop-cac-doan-...,Tổng hợp các đoạn đường bị ngập nước vào mùa m...,function goBack() { window.history.back(); } C...


In [ ]:
print(df.to_string())

                                                                                                                url                                                                                        title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
# Tải file excel kết quả về gdrive
df.to_excel(r'drive/MyDrive/Do an cuoi ky/flooded_street_rawdata.xlsx', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
